In [ ]:
import deeplabcut as dlc
import random
import shutil
import glob
import os

# From paths.py
import paths

In [ ]:
# Find all videos in the data directory with VTuning in the name
all_videos = glob.glob(paths.vrexperiment_path + os.sep + "*VTuningWF*.avi")

# If Project already created, run the cell below

In [ ]:
# Get config path
# config_path = paths.config_path_VTuningWF
config_path = r"D:\DLC_projects\vtuning_wirefree_DLC_22\VTuningWF-Matt-2022-10-25\config.yaml"

# Read config file
cfg = dlc.auxiliaryfunctions.read_config(config_path)

# Get project directory
project_directory = cfg['project_path']

# Get list of local videos as found in the config
project_video_list = [vid for vid in cfg["video_sets"]]
test_video_dir = os.path.join(project_directory, "test-videos")
test_vids = glob.glob(test_video_dir + os.sep + "*.avi")
test_vids += glob.glob(test_video_dir + os.sep + "*.mp4")
test_vids = [vid for vid in test_vids if "labeled" not in vid]

# If you haven't created a new project, run this cell

In [ ]:
dlc.create_new_project?

In [ ]:
# Randomly select 15 videos to train on
remote_video_list = []
for i in range(4):
    remote_video_list.append(random.choice(all_videos))

dlc.create_new_project("VTuningWF", "Matt", remote_video_list, working_directory=r"D:\DLC_projects\vtuning_wirefree", copy_videos=True)

In [ ]:
# Get config and path info
config_path = paths.config_path_VTuningWF
cfg = dlc.auxiliaryfunctions.read_config(config_path)
project_directory = cfg['project_path']
project_video_list = [vid for vid in cfg["video_sets"]]

In [ ]:
# Randomly pick videos to test the network that are not part of the training set and copy to the project directory
test_video_dir = os.path.join(project_directory, "test-videos")
if not os.path.isdir(test_video_dir):
    os.mkdir(test_video_dir)

# Randomly pick five videos outside of the training set to test the network on
test_vids = []
while(len(test_vids) < 5):
    vid = random.choice(all_videos)
    if vid not in project_video_list:
        dest = shutil.copy2(vid, test_video_dir)
        test_vids.append(dest)
        
test_vids = glob.glob(test_video_dir + os.sep + "*VTuningWF*.avi")

In [ ]:
test_vids

# Add extra videos (if needed)

In [ ]:
dlc.add_new_videos?

In [ ]:
# If adding new videos, do that here by randomly selecting some not in the original data set or the test set
# For this particular case, we want videos that are older than March 12th 2022
from datetime import datetime
from os.path import sep

valid_extras = [vid for vid in all_videos if datetime.strptime(vid.split(sep)[-1].split('_VTuningWF')[0], "%m_%d_%Y_%H_%M_%S") > datetime(2022, 3, 12)]

extra_vids = []
while(len(extra_vids) < 2):
    vid = random.choice(valid_extras)
    if vid not in project_video_list or test_vids:
        extra_vids.append(vid)
        
dlc.add_new_videos(config_path, extra_vids, copy_videos=True)

# Extract and label frames

In [ ]:
dlc.extract_frames?

In [ ]:
# extract frames from the videos
# will go through all of the videos on the list, so if many videos, it might take a while
dlc.extract_frames(config_path, mode='automatic', algo='kmeans', crop=False, userfeedback=True);

In [ ]:
# label frames
# generates folders in the labeled-data folder with the video name
dlc.label_frames(config_path)

In [ ]:
# check the labels
# generates folders in the labeled-data folder with the video name but the actual labels printed on the pics
dlc.check_labels(config_path, draw_skeleton=False);

In [ ]:
# Used if need to |reconstruct labeling h5 files from csvs
dlc.convertcsv2h5(config_path)

In [ ]:
dlc.create_training_dataset?

In [ ]:
# create the training set
# also creates the Pose.yaml in the dlc-models/train dataset, which contains the setting for training, i.e. take a look
dlc.create_training_dataset(config_path, num_shuffles=3,
                            augmenter_type='imgaug',
                            net_type='resnet_50',
                            posecfg_template=r"D:\DLC_projects\vtuning_wirefree_DLC_22\VTuningWF-Matt-2022-10-25\pose_cfg.yaml"
                            );

In [ ]:
dlc.train_network?

In [ ]:
# train the network
dlc.train_network(config_path, shuffle=2,
                  gputouse=0, allow_growth=True,
                  max_snapshots_to_keep=5, displayiters=5e3, saveiters=25e3, maxiters=200e3)

In [ ]:
# evaluate the network|
dlc.evaluate_network(config_path, Shuffles=[2], plotting=True)

In [ ]:
dlc.analyze_videos?

In [ ]:
test_vids

In [ ]:
dlc.analyze_videos(config_path, [test_vids[-1]], shuffle=2, dynamic=(True, .1, 50), gputouse=0)

In [ ]:
dlc.filterpredictions?

In [ ]:
dlc.filterpredictions(config_path, [test_vids[-1]], shuffle=2, filtertype='arima', p_bound=0.01, ARdegree=3, MAdegree=1, alpha=0.01)

In [ ]:
dlc.plot_trajectories?

In [ ]:
# Plot tracjectories
%matplotlib notebook
dlc.plot_trajectories(config_path, [test_vids[-1]], shuffle=2, showfigures=True)

In [ ]:
dlc.create_labeled_video?

In [ ]:
dlc.create_labeled_video(config_path, [test_vids[-1]], shuffle=2, filtered=True, draw_skeleton=True)

In [ ]:
dlc.extract_outlier_frames?

In [ ]:
# extract outlier frames
# dlc.extract_outlier_frames(config_path, project_video_list, outlieralgorithm='uncertain', p_bound=0.6)
dlc.extract_outlier_frames(config_path, project_video_list,
                          outlieralgorithm='jump', epsilon=8)

In [ ]:
dlc.refine_labels(config_path)

In [ ]:
# Merge the dataset, then move to check labels and retrain
dlc.merge_datasets(config_path)